#### AGENT
An agent take a self-determined, input-dependent sequence of steps before returning a user-facing output

First: Build an agent that has two tools: one to look things up online (Tavily), and one to look up specific data that we’ve loaded into a index using a retriever.

In [1]:
# Libraries

%pip install langchain
%pip install --upgrade --quiet  langchain-openai
%pip install langchainhub
%pip install BeautifulSoup4
%pip install faiss-cpu

  Obtaining dependency information for langchain-core<0.2.0,>=0.1.31 from https://files.pythonhosted.org/packages/43/8b/48b7e6de9041d2b33d5108e154b82d1bd6c47cc68f0e44cb4fcdaccf5ec7/langchain_core-0.1.52-py3-none-any.whl.metadata
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 5.9 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.3
    Uninstalling langchain-core-0.2.3:
      Successfully uninstalled langchain-core-0.2.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.8 requires langchain-core<0.3,>=0.2.2, but you have langchain-core 0.1.52 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to 

In [2]:
%load_ext dotenv
%dotenv

#### Tavily Search engine tool

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()
search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1717525578, 'localtime': '2024-06-04 11:26'}, 'current': {'last_updated_epoch': 1717524900, 'last_updated': '2024-06-04 11:15', 'temp_c': 15.0, 'temp_f': 59.0, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 15.0, 'wind_kph': 24.1, 'wind_degree': 330, 'wind_dir': 'NNW', 'pressure_mb': 1014.0, 'pressure_in': 29.95, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 82, 'cloud': 0, 'feelslike_c': 14.7, 'feelslike_f': 58.4, 'windchill_c': 18.3, 'windchill_f': 64.9, 'heatindex_c': 18.3, 'heatindex_f': 64.9, 'dewpoint_c': 10.6, 'dewpoint_f': 51.1, 'vis_km': 6.4, 'vis_miles': 3.0, 'uv': 5.0, 'gust_mph': 19.5, 'gust_kph': 31.3}}"},
 {'url': 'https://www.timeanddate.com/weather/usa/san

#### Retriever over data

In [6]:
# FAISS (Facebook AI Similarity Search) is a library that allows developers to quickly search for embeddings of documents that are similar to each other.

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool


loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)

embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002", openai_api_version="2023-05-15")
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

retriever.invoke("how to upload a dataset")[0]

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [7]:
# Tool

tools = [search, retriever_tool]

In [18]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment="gpt-4o",
)

In [19]:
from langchain import hub

# Get the prompt to use - you can modify this!

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [20]:
# Initialize an agent

from langchain.agents import create_openai_functions_agent

agent = create_openai_functions_agent(llm, tools, prompt)

In [21]:
# AgentExecutor - repeatedly call the agent and execute tools

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [22]:
agent_executor.invoke({"input": "hi!"})



> Entering new AgentExecutor chain...
Hello! How can I assist you today?

> Finished chain.


{'input': 'hi!', 'output': 'Hello! How can I assist you today?'}

In [23]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can langsmith help with testing'}`




Error in LangChainTracer.on_tool_end callback: TracerException("Found chain run at ID 8307c51f-09fb-4046-8786-9c4ae384a3b8, but expected {'tool'} run.")


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​We provide multiple ways to log traces to LangSmith. Below, we'll highlight

"revision_id": "beta"    },)impo

{'input': 'how can langsmith help with testing?',
 'output': "LangSmith is a platform designed to facilitate the building and testing of production-grade LLM (Large Language Model) applications. Here are some ways LangSmith can help with testing:\n\n1. **Trace Logging**: LangSmith allows you to log traces of your application's interactions with LLMs. This helps in monitoring and evaluating the performance and behavior of your application in real-time.\n\n2. **Dataset Creation**: You can create datasets that represent your test cases. This involves defining inputs and expected outputs, which can be used to systematically test your application.\n\n3. **Evaluation**: LangSmith provides tools for evaluating your application against these datasets. You can define evaluators, such as exact match evaluators, to automatically compare the actual outputs of your application with the expected outputs.\n\n4. **API Integration**: LangSmith integrates with various APIs, including OpenAI, allowing yo